## **Boundary conditions ... <font color='blue'> as a list of boundaries, </font><font color='red'> as a numpy recarray, </font> <font color='green'> or as a dictionary! </font>**

In [1]:
import os ; import numpy as np ; import flopy; from pathlib import Path

In [2]:
sim_name = Path("./01_mf_boundaries_tutorial") ; sim_name.mkdir(exist_ok=True) ;  modelname = "B_test"
exe_name = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe'                    ; workspace  = output_folder = model_ws = ws = sim_name

In [3]:
stress_period_data = [ [2,3,4,  10.7, 5000.0, -5.7,],  
                       [2,3,5,  10.7, 5000.0, -5.7,],
                       [2,3,6,  10.7, 5000.0, -5.7,],]                  # layer, row, column,        stage, conductance, river bottom

In [4]:
riv_dtype = flopy.modflow.ModflowRiv.get_default_dtype()   ; print(riv_dtype)             # Recarrays

[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('stage', '<f4'), ('cond', '<f4'), ('rbot', '<f4')]


In [5]:
stress_period_data = np.zeros((3), dtype=riv_dtype)   ;   stress_period_data = stress_period_data.view(np.recarray)
print("stress_period_data: ", stress_period_data, "type is: ", type(stress_period_data))

stress_period_data:  [(0, 0, 0, 0., 0., 0.) (0, 0, 0, 0., 0., 0.) (0, 0, 0, 0., 0., 0.)] type is:  <class 'numpy.recarray'>


In [6]:
stress_period_data[0] = (2, 3, 4, 10.7, 5000.0, -5.7)  
stress_period_data[1] = (2, 3, 5, 10.7, 5000.0, -5.7)
stress_period_data[2] = (2, 3, 6, 10.7, 5000.0, -5.7)          ; print(stress_period_data)

[(2, 3, 4, 10.7, 5000., -5.7) (2, 3, 5, 10.7, 5000., -5.7)
 (2, 3, 6, 10.7, 5000., -5.7)]


In [7]:
sp1 = [ [2,3,4,  10.7, 5000.0, -5.7,],  [2,3,5,  10.7, 5000.0, -5.7,], [2,3,6,  10.7, 5000.0, -5.7]]; sp1

[[2, 3, 4, 10.7, 5000.0, -5.7],
 [2, 3, 5, 10.7, 5000.0, -5.7],
 [2, 3, 6, 10.7, 5000.0, -5.7]]

In [8]:
riv_dtype = flopy.modflow.ModflowRiv.get_default_dtype()
sp5 = np.zeros((3), dtype=riv_dtype)    ; sp5 = sp5.view(np.recarray)
sp5[0] = (2, 3, 4, 20.7, 5000.0, -5.7)  ; sp5[1] = (2, 3, 5, 20.7, 5000.0, -5.7)  ; sp5[2] = (2, 3, 6, 20.7, 5000.0, -5.7)

sp_dict = {0: 0, 1: sp1, 2: 0, 5: sp5}
m   = flopy.modflow.Modflow    (modelname=modelname, model_ws=ws )
riv = flopy.modflow.ModflowRiv (m, stress_period_data=sp_dict )
m.write_input()

## **riva_dtype**  ... string in the last column of the list in order to name each boundary condition.

In [9]:
# create an empty array with an iface auxiliary variable at the end
riva_dtype = [("k", "<i8"),("i", "<i8"),("j", "<i8"),("stage", "<f4"),("cond", "<f4"),("rbot", "<f4"),("iface", "<i4"),("boundname", object)]

riva_dtype = np.dtype(riva_dtype)     ;stress_period_data = np.zeros((3),dtype=riva_dtype)   ; stress_period_data = stress_period_data.view(np.recarray)
print("stress_period_data: ", stress_period_data)  ; print("type is: ", type(stress_period_data))

stress_period_data[0] = (2, 3, 4, 10.7, 5000.0, -5.7, 1, "riv1")
stress_period_data[1] = (2, 3, 5, 10.7, 5000.0, -5.7, 2, "riv2")
stress_period_data[2] = (2, 3, 6, 10.7, 5000.0, -5.7, 3, "riv3")

m = flopy.modflow.Modflow(modelname="test", model_ws=workspace)
riv = flopy.modflow.ModflowRiv(m ,stress_period_data=stress_period_data  ,dtype=riva_dtype   ,options=["aux iface"])
m.write_input()

stress_period_data:  [(0, 0, 0, 0., 0., 0., 0, 0) (0, 0, 0, 0., 0., 0., 0, 0)
 (0, 0, 0, 0., 0., 0., 0, 0)]
type is:  <class 'numpy.recarray'>


## **rivu_dtype**  MODFLOW-USG

In [10]:
rivu_dtype = [("nodenumber", "<i8"), ("stage", "<f4"), ("cond", "<f4"), ("rbot", "<f4"),]     
rivu_dtype = np.dtype(rivu_dtype)  ; stress_period_data = np.zeros((3), dtype=rivu_dtype) ; stress_period_data = stress_period_data.view(np.recarray)
stress_period_data[0] = (77, 10.7, 5000.0, -5.7)
stress_period_data[1] = (245, 10.7, 5000.0, -5.7)
stress_period_data[2] = (450034, 10.7, 5000.0, -5.7)

m = flopy.modflow.Modflow(modelname=modelname, model_ws=ws)
riv = flopy.modflow.ModflowRiv(m, stress_period_data=stress_period_data   ,dtype=rivu_dtype)
m.write_input()

## **Combining** two boundary condition packages

In [12]:
ml       = flopy.modflow.Modflow(modelname=modelname, model_ws=ws)     ; dis = flopy.modflow.ModflowDis(ml, 10, 10, 10, 10) # AUX
sp_data1 = {3: [1, 1, 1, 1.0], 5: [1, 2, 4, 4.0]}                    
wel1     = flopy.modflow.ModflowWel(ml, stress_period_data = sp_data1)  # ml.write_input()        
sp_data2 = {0: [1, 1, 3, 3.0], 8: [9, 2, 4, 4.0]}
wel2     = flopy.modflow.ModflowWel(ml, stress_period_data=sp_data2)        # ml.write_input()
wel3 = flopy.modflow.ModflowWel(ml, stress_period_data = wel2.stress_period_data.append(wel1.stress_period_data),)  
ml.write_input()